In [4]:
# !pip install bm3d
import bm3d

In [5]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import os
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, random_split

In [6]:
def minMaxNormalized(image):
    return cv2.normalize(image, None, 0, 1, cv2.NORM_MINMAX,dtype=cv2.CV_32F)

In [7]:
def colorSeparator(image_rgb):
    r = image_rgb[:,:,0]
    g = image_rgb[:,:,1]
    b = image_rgb[:,:,2]
    return r, g, b

In [9]:
# return red, green and blue freq. domain
def to_frequencyDomainGray(gray_img):
    normalized     = gray_img#minMaxNormalized(gray_img)
    norm_fft       = np.fft.fft2(normalized)
    norm_fft_shift = np.fft.fftshift(norm_fft)
    return norm_fft_shift

def to_frequencyDomain(image):
    r,g,b = colorSeparator(image)
    r_fftshft = to_frequencyDomainGray(r)
    b_fftshft = to_frequencyDomainGray(b)
    g_fftshft = to_frequencyDomainGray(g)
    return r_fftshft, b_fftshft, g_fftshft

def magnitudeLog(img):
    mag = np.sqrt(img.real ** 2 + img.imag ** 2)
    return np.log1p(mag)

In [10]:
def to_spatialDomain(image):
    image_ishift = np.fft.ifftshift(image)
    image_ifft   = np.fft.ifft2(image_ishift)
    image_real   = np.abs(image_ifft)
    # denormlized  = image * 255
    return image_real

In [11]:
def reduceSaltAndPeperNoise(image, kernel=3):
    image = np.array(image)
    median_filtered = cv2.medianBlur(image, kernel)
    return Image.fromarray(median_filtered)

In [12]:
#use bm3d method. before use this function make sure install package. (pip3 install bm3d)
def reduceGaussianNoise(image, sigma=20/255):
    image     = np.array(image)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    R, G, B   = cv2.split(image_rgb)

    R_denoised = bm3d.bm3d(R / 255.0, sigma) * 255
    G_denoised = bm3d.bm3d(G / 255.0, sigma) * 255
    B_denoised = bm3d.bm3d(B / 255.0, sigma) * 255

    image_denoised = cv2.merge((R_denoised, G_denoised, B_denoised))
    image_denoised = np.clip(image_denoised, 0, 255).astype(np.uint8)

    image_denoised = cv2.cvtColor(image_denoised, cv2.COLOR_RGB2BGR)

    return image_denoised

In [ ]:
# 1. خواندن فایل CSV و فیلتر کردن تصاویر periodic
df = pd.read_csv("/content/drive/MyDrive/noise reduce/Labels.csv")

# فیلتر کردن فقط تصاویری که noise_type = periodic دارند
periodic_images = df[df["noise_type"] == "Periodic"]["image_name"].tolist()

In [ ]:
class DenoisingDatasetPeriodic(Dataset):
    def __init__(self, noisy_dir, clean_dir, image_list, transform=None):
        self.noisy_dir = noisy_dir
        self.clean_dir = clean_dir
        self.transform = transform

        # فیلتر کردن تصاویر نامعتبر
        self.image_list = []
        for filename in image_list:
            noisy_path = os.path.join(self.noisy_dir, filename)
            clean_path = os.path.join(self.clean_dir, filename)

            if os.path.exists(noisy_path) and os.path.exists(clean_path):
                noisy_img = cv2.imread(noisy_path)
                clean_img = cv2.imread(clean_path)

                if noisy_img is not None and clean_img is not None:
                    self.image_list.append(filename)

        print(f"✅ تعداد {len(self.image_list)} تصویر معتبر از {len(image_list)} تصویر انتخاب شد.")

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        filename = self.image_list[idx]
        noisy_path = os.path.join(self.noisy_dir, filename)
        clean_path = os.path.join(self.clean_dir, filename)

        noisy_img = cv2.imread(noisy_path)
        clean_img = cv2.imread(clean_path)

        # تبدیل BGR به RGB
        noisy_img = cv2.cvtColor(noisy_img, cv2.COLOR_BGR2RGB)
        clean_img = cv2.cvtColor(clean_img, cv2.COLOR_BGR2RGB)

        # تغییر اندازه تصویر
        noisy_img = cv2.resize(noisy_img, (256, 256))
        clean_img = cv2.resize(clean_img, (256, 256))
        # noisy_img = cv2.resize(noisy_img, (512, 384))
        # clean_img = cv2.resize(clean_img, (512, 384)) #out memory

        if self.transform:
            noisy_img = self.transform(noisy_img)
            clean_img = self.transform(clean_img)

        return noisy_img, clean_img

# 3. تنظیمات پردازش تصویر برای RGB
transformP = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

In [ ]:
import random
# تقسیم 80٪ برای آموزش و 20٪ برای تست
random.shuffle(periodic_images)
train_size = int(0.9 * len(periodic_images))
test_size  = len(periodic_images) - train_size
train_images, test_images = periodic_images[:train_size], periodic_images[train_size:]
# 4. ساخت دیتاست‌های آموزش و تست
train_dataset = DenoisingDatasetPeriodic("/content/drive/MyDrive/noise reduce/Noisy", "/content/drive/MyDrive/noise reduce/Clean", train_images, transform=transform)
test_dataset  = DenoisingDatasetPeriodic("/content/drive/MyDrive/noise reduce/Noisy", "/content/drive/MyDrive/noise reduce/Clean", test_images, transform=transform)
train_loader  = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader   = DataLoader(test_dataset, batch_size=4, shuffle=False)

In [ ]:

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        # ✅ فقط یک بار Encoder تعریف شود
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1),  # ورودی RGB
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)  # در اینجا ابعاد تصویر کاهش یافته
        )

        # ✅ فقط یک بار Decoder تعریف شود
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 3, 3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()  # خروجی در محدوده [0,1]
        )

    def forward(self, x):
        x = self.encoder(x)
        # print(f"Encoder Output Shape: {x.shape}")  # 🚀 بررسی خروجی Encoder
        x = self.decoder(x)
        # print(f"Decoder Output Shape: {x.shape}")  # 🚀 بررسی خروجی Decoder
        return x

# تست مدل
encoder      = Autoencoder()
sample_input = torch.randn(1, 3, 64, 64)  # یک تصویر RGB با سایز 64x64
output       = encoder(sample_input)

# 6. تنظیم مدل و آموزش
device    = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder   = Autoencoder().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(encoder.parameters(), lr=0.0005)

In [ ]:
# 7. آموزش مدل روی دیتاست Train
# num_epochs = 50
num_epochs = 30
for epoch in range(num_epochs):
    encoder.train()
    for noisy_imgs, clean_imgs in train_loader:
        noisy_imgs, clean_imgs = noisy_imgs.to(device), clean_imgs.to(device)

        optimizer.zero_grad()
        outputs = encoder(noisy_imgs)
        loss = criterion(outputs, clean_imgs)
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

In [ ]:
# 8. ارزیابی مدل روی دیتاست Test
encoder.eval()
total_loss = 0
with torch.no_grad():
    for noisy_imgs, clean_imgs in test_loader:
        noisy_imgs, clean_imgs = noisy_imgs.to(device), clean_imgs.to(device)
        outputs = encoder(noisy_imgs)
        loss    = criterion(outputs, clean_imgs)
        total_loss += loss.item()

avg_test_loss = total_loss / len(test_loader)
print(f"Test Loss: {avg_test_loss:.4f}")

In [ ]:
def reducePeriodicNoise(image, model, transform, device):
    model.eval()

    # خواندن تصویر با OpenCV (BGR) و تبدیل به RGB
    noisy_img = cv2.imread(image)
    noisy_img = cv2.cvtColor(noisy_img, cv2.COLOR_BGR2RGB)

    # تبدیل به یک تصویر PIL
    noisy_img = Image.fromarray(noisy_img)

    # اعمال transform
    noisy_tensor = transform(noisy_img).unsqueeze(0).to(device)

    with torch.no_grad():
        denoised_tensor = model(noisy_tensor).cpu().squeeze(0)

    # تبدیل خروجی مدل از Tensor به NumPy
    denoised_img = denoised_tensor.permute(1, 2, 0).numpy()  # (C, H, W) → (H, W, C)

    # از Normalization خارج شود (برگرداندن مقادیر به محدوده [0,255])
    denoised_img = (denoised_img * 255).clip(0, 255).astype(np.uint8)

    return denoised_img

In [16]:
# maybe we need this function
def psnr(original_image, reconstructed_image):
    original_image      = np.array(original_image, dtype=np.float64) 
    reconstructed_image = np.array(reconstructed_image, dtype=np.float64) 

    mse = np.mean((original_image - reconstructed_image) ** 2)

    if mse == 0:
        return float('inf')

    max_pixel_value = 255
    psnr = 10 * np.log10((max_pixel_value ** 2) / mse)
    return psnr

In [20]:
from pathlib import Path

class NoiseDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.data       = pd.read_csv(csv_file)
        self.img_dir    = img_dir
        self.transform  = transform
        self.label_map  = {"Salt & Pepper": 0, "Gaussian": 1, "Periodic": 2}
        self.valid_data = self._filter_valid_images()

    def _filter_valid_images(self):
        valid_rows = []
        for idx in range(len(self.data)):
            img_path = Path(self.img_dir) / self.data.iloc[idx, 0]
            if img_path.exists():  # بررسی اینکه آیا تصویر موجود است
                valid_rows.append(self.data.iloc[idx])
        return pd.DataFrame(valid_rows)

    def __len__(self):
        return len(self.valid_data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.valid_data.iloc[idx, 0])
        image = Image.open(img_name).convert("RGB")
        label = self.label_map[self.valid_data.iloc[idx, 1]]

        if self.transform:
            image = self.transform(image)

        return image, label

In [21]:
transform = transforms.Compose([
    transforms.Resize((256, 192)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# ایجاد دیتاست
csv_path = "/content/drive/MyDrive/reduce noise/Labels.csv"
img_dir  = "/content/drive/MyDrive/reduce noise/Noisy"
full_dataset = NoiseDataset(csv_path, img_dir, transform=transform)

# تقسیم ۸۰٪ آموزش و ۲۰٪ تست
train_size = int(0.8 * len(full_dataset))
test_size  = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# DataLoader ها
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [22]:
class CNNModel(nn.Module): #CNN definition
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool  = nn.MaxPool2d(2, 2)
        self.fc1   = nn.Linear(64 * 32 * 24, 128)
        self.fc2   = nn.Linear(128, 3)
        self.relu  = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))

        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [23]:
device    = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model     = CNNModel().to(device)
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-4)

In [26]:
#train model
epochs    = 15
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct      = 0
    total        = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # calcute prediction
        _, predicted = torch.max(outputs, 1)
        total   += labels.size(0)
        correct += (predicted == labels).sum().item()

        running_loss += loss.item()

    scheduler.step()

    train_acc = 100 * correct / total
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_acc:.2f}%")


Epoch 1, Loss: 0.1325, Train Acc: 96.75%
Epoch 2, Loss: 0.1417, Train Acc: 95.83%
Epoch 3, Loss: 0.1319, Train Acc: 96.82%
Epoch 4, Loss: 0.1324, Train Acc: 95.89%
Epoch 5, Loss: 0.1366, Train Acc: 95.89%
Epoch 6, Loss: 0.1277, Train Acc: 96.16%
Epoch 7, Loss: 0.1429, Train Acc: 96.56%
Epoch 8, Loss: 0.1340, Train Acc: 96.42%
Epoch 9, Loss: 0.1358, Train Acc: 96.69%
Epoch 10, Loss: 0.1458, Train Acc: 95.56%
Epoch 11, Loss: 0.1406, Train Acc: 96.36%
Epoch 12, Loss: 0.1343, Train Acc: 96.16%
Epoch 13, Loss: 0.1319, Train Acc: 96.56%
Epoch 14, Loss: 0.1409, Train Acc: 96.09%
Epoch 15, Loss: 0.1337, Train Acc: 96.36%


In [27]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        y_true.extend(labels.cpu().numpy())  # تبدیل به لیست برای sklearn
        y_pred.extend(predicted.cpu().numpy())

# محاسبه معیارهای ارزیابی
accuracy    = accuracy_score  (y_true, y_pred)
precision   = precision_score (y_true, y_pred, average="weighted")
recall      = recall_score    (y_true, y_pred, average="weighted")
f1          = f1_score        (y_true, y_pred, average="weighted")
conf_matrix = confusion_matrix(y_true, y_pred)

# محاسبه Specificity
tn = conf_matrix[0][0]  # True Negative
fp = conf_matrix[0][1]  # False Positive
specificity = tn / (tn + fp) if (tn + fp) != 0 else 0  # جلوگیری از تقسیم بر صفر

# نمایش نتایج
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"Specificity: {specificity:.2f}")
print(f"F1-Score: {f1:.2f}")
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 87.50%


In [25]:
# کلاس‌های نویز
class_labels = ["Salt & Pepper", "Gaussian", "Periodic"]

# مسیر پوشه ورودی (تصاویر تست) و خروجی (تصاویر فیلتر شده)
image_folder       = "/content/drive/MyDrive/reduce noise/Test" #NoisyTest
output_folder      = "/content/drive/MyDrive/reduce noise/filtered_images" 
clean_image_folder = "/content/drive/MyDrive/reduce noise/Clean" #Without-NoisyTest

# اگر پوشه خروجی وجود ندارد، ایجاد شود
os.makedirs(output_folder, exist_ok=True)


# مقداردهی اولیه لیست برای ذخیره مقادیر PSNR
psnr_values = []

for img_name in os.listdir(image_folder):
    img_path = os.path.join(image_folder, img_name)
    image    = Image.open(img_path).convert("RGB")  # خواندن تصویر و تبدیل به RGB
    image_tensor = transform(image)  # پیش‌پردازش
    image_tensor = image_tensor.unsqueeze(0).to(device)  # اضافه کردن بعد batch و ارسال به GPU/CPU

    # پیش‌بینی مدل
    with torch.no_grad():
        output       = model(image_tensor)
        _, predicted = torch.max(output, 1)  # پیدا کردن کلاس با بالاترین احتمال

    predicted_label  = class_labels[predicted.item()]

    # انتخاب فیلتر مناسب بر اساس نوع نویز پیش‌بینی شده
    if predicted_label == "Salt & Pepper":
        filtered_image = reduceSaltAndPeperNoise(image)
    elif predicted_label == "Gaussian":
        filtered_image = reduceGaussianNoise(image)
    elif predicted_label == "Periodic":
        filtered_image = reducePeriodicNoise(image, encoder, transformP, device)
        
    # مسیر ذخیره تصویر فیلتر شده
    output_path = os.path.join(output_folder, img_name)
    filtered_image.save(output_path)

    # بررسی اینکه آیا تصویر اصلی (clean) موجود است یا نه
    original_path = os.path.join(clean_image_folder, img_name)
    if os.path.exists(original_path):
        original_image = Image.open(original_path).convert("RGB")
        psnr_value = psnr(original_image, filtered_image)  # محاسبه PSNR
        psnr_values.append(psnr_value)  # ذخیره مقدار PSNR در لیست

# محاسبه و نمایش میانگین PSNR
if psnr_values:
    mean_psnr = np.mean(psnr_values)
    print(f"میانگین PSNR برای تمام تصاویر: {mean_psnr:.2f} dB")
else:
    print("هیچ مقدار PSNR محاسبه نشد!")

In [ ]:
# مسیر پوشه ورودی (تصاویر تست) و خروجی (تصاویر فیلتر شده)
image_folder       = "/content/drive/MyDrive/reduce noise/NoisyTest"
output_folder      = "/content/drive/MyDrive/reduce noise/filtered_images"
clean_image_folder = "/content/drive/MyDrive/reduce noise/Without-NoisyTest"

# اگر پوشه خروجی وجود ندارد، ایجاد شود
os.makedirs(output_folder, exist_ok=True)


# مقداردهی اولیه لیست برای ذخیره مقادیر PSNR
psnr_values = []

for img_name in os.listdir(image_folder):
    img_path = os.path.join(image_folder, img_name)
    image = Image.open(img_path).convert("RGB")  # خواندن تصویر و تبدیل به RGB
    image_tensor = transform(image)  # پیش‌پردازش
    image_tensor = image_tensor.unsqueeze(0).to(device)  # اضافه کردن بعد batch و ارسال به GPU/CPU

    # پیش‌بینی مدل
    with torch.no_grad():
        output = model(image_tensor)
        _, predicted = torch.max(output, 1)  # پیدا کردن کلاس با بالاترین احتمال

    predicted_label = class_labels[predicted.item()]

    # انتخاب فیلتر مناسب بر اساس نوع نویز پیش‌بینی شده
    if predicted_label == "Salt & Pepper":
        filtered_image = reduceSaltAndPeperNoise(image)
    elif predicted_label == "Gaussian":
        filtered_image = reduceGaussianNoise(image)
    elif predicted_label == "Periodic":
        filtered_image = reducePeriodicNoise(image)

    # مسیر ذخیره تصویر فیلتر شده
    output_path = os.path.join(output_folder, img_name)
    filtered_image.save(output_path)

    # بررسی اینکه آیا تصویر اصلی (clean) موجود است یا نه
    original_path = os.path.join(clean_image_folder, img_name)
    if os.path.exists(original_path):
        original_image = Image.open(original_path).convert("RGB")
        psnr_value = psnr(original_image, filtered_image)  # محاسبه PSNR
        psnr_values.append(psnr_value)  # ذخیره مقدار PSNR در لیست

# محاسبه و نمایش میانگین PSNR
if psnr_values:
    mean_psnr = np.mean(psnr_values)
    print(f"میانگین PSNR برای تمام تصاویر: {mean_psnr:.2f} dB")
else:
    print("هیچ مقدار PSNR محاسبه نشد!")